In [ ]:
import torch
from PIL import Image
from transformers import BlipProcessor, BlipForConditionalGeneration
import nltk
from nltk.tokenize import TreebankWordTokenizer, word_tokenize
from nltk import pos_tag
from gtts import gTTS
import gradio as gr
import os
import emoji
from deep_translator import GoogleTranslator


for resource in ["averaged_perceptron_tagger", "averaged_perceptron_tagger_eng", "punkt"]:
    try:
        nltk.data.find(f"taggers/{resource}")
    except LookupError:
        nltk.download(resource, quiet=True)

_tokenizer = TreebankWordTokenizer()


pos_map = {
    "NN": ("Noun 🧸", "#ADD8E6"),
    "NNS": ("Noun 🧸", "#ADD8E6"),
    "NNP": ("Proper Noun 👤", "#90EE90"),
    "JJ": ("Adjective 🎨", "#FFB6C1"),
    "VB": ("Verb 🏃", "#FFFF99"),
    "VBD": ("Verb 🏃", "#FFFF99"),
    "VBG": ("Verb 🏃", "#FFFF99"),
    "VBN": ("Verb 🏃", "#FFFF99"),
    "VBP": ("Verb 🏃", "#FFFF99"),
    "VBZ": ("Verb 🏃", "#FFFF99"),
    "RB": ("Adverb 🌀", "#FFA500"),
    "DT": ("Article 📄", "#D3D3D3"),
    "PRP": ("Pronoun 🙋", "#EE82EE"),
    "IN": ("Preposition ➡️", "#E0FFFF"),
    "CC": ("Conjunction 🔗", "#E6E6FA")
}

pos_map_ar = {
    "ال": ("تعريف 📘", "#FFDAB9"),
    "و": ("حرف عطف 🔗", "#E6E6FA"),
    "في": ("حرف جر ➡️", "#E0FFFF"),
    "من": ("حرف جر ➡️", "#E0FFFF"),
    "على": ("حرف جر ➡️", "#E0FFFF"),
    "عن": ("حرف جر ➡️", "#E0FFFF"),
    "إلى": ("حرف جر ➡️", "#E0FFFF"),
}

def analyze_pos_child_friendly(text, lang="en"):
    if lang == "en":
        tokens = _tokenizer.tokenize(text)
        tagged = pos_tag(tokens)
        html = "<table style='border-collapse: collapse; width: 100%; border: 2px solid #000;'>"
        html += "<tr style='background-color:#FFD700; font-size:16px;'><th>Word</th><th>POS</th></tr>"
        for word, tag in tagged:
            label, color = pos_map.get(tag, ("Word", "white"))
            html += f"<tr><td style='border:2px solid #000;padding:8px;background-color:#F0F8FF;'>{word}</td>"
            html += f"<td style='border:2px solid #000;padding:8px;background-color:{color}'>{label}</td></tr>"
        html += "</table>"
        return html
    else:
        tokens = word_tokenize(text)
        html = "<table style='border-collapse: collapse; width: 100%; border: 2px solid #000;'>"
        html += "<tr style='background-color:#98FB98; font-size:16px;'><th>الكلمة</th><th>النوع</th></tr>"
        for word in tokens:
            label, color = pos_map_ar.get(word, ("كلمة", "white"))
            html += f"<tr><td style='border:2px solid #000;padding:8px;background-color:#F0FFF0;'>{word}</td>"
            html += f"<td style='border:2px solid #000;padding:8px;background-color:{color}'>{label}</td></tr>"
        html += "</table>"
        return html


CHECKPOINT_DIR = "/content/drive/MyDrive/Checkpoints1"
ckpts = sorted([c for c in os.listdir(CHECKPOINT_DIR) if c.startswith("checkpoint-")])
last_ckpt = os.path.join(CHECKPOINT_DIR, ckpts[-1])

device = "cuda" if torch.cuda.is_available() else "cpu"
model = BlipForConditionalGeneration.from_pretrained(last_ckpt).to(device)
processor = BlipProcessor.from_pretrained(last_ckpt)


def generate_story(image):
    inputs = processor(images=image, return_tensors="pt").to(device)
    output_ids = model.generate(**inputs, max_length=60, num_beams=5)
    story = processor.decode(output_ids[0], skip_special_tokens=True)
    return story

def get_audio(story_text, lang='en'):
    tts = gTTS(text=story_text, lang=lang)
    filename = "story.mp3" if lang == "en" else "story_ar.mp3"
    tts.save(filename)
    return filename

def translate_to_arabic(text):
    if not text.strip():
        return "⚠️ لا يوجد نص للترجمة"
    translation = GoogleTranslator(source="en", target="ar").translate(text)
    return translation


css_html = """
<style>
#small_btn {
    width: 40px !important;
    height: 40px !important;
    padding: 5px !important;
    font-size: 16px !important;
}
#small_dropdown {
    width: 80px !important;
    font-size: 14px !important;
}
</style>
"""


with gr.Blocks() as demo:
    gr.HTML(css_html)

    gr.Markdown("<h1 style='color:#FF69B4;text-align:center'>🌟 Fun Story Generator for Kids 🌟</h1>")
    gr.Markdown("<p style='color:#1E90FF;font-size:18px;text-align:center'>Upload an image, get a story, see the parts of speech, and listen!</p>")

    with gr.Row():
        with gr.Column():
            gr.Markdown("🖼️ Upload your image")
            image_input = gr.Image(type="pil", label="")

            with gr.Row():
                story_output = gr.HTML(
                    "<div style='border:5px solid #FF1493; border-radius:20px; padding:10px; background-color:#E0FFFF; min-height:120px;'>Your story will appear here...</div>"
                )
                gen_audio_btn = gr.Button("🔊", elem_id="small_btn")

            with gr.Row():
                story_ar_output = gr.HTML(
                    "<div style='border:5px solid #32CD32; border-radius:20px; padding:10px; background-color:#F0FFF0; min-height:120px;'>اضغط زر الترجمة لترى القصة بالعربية...</div>"
                )
                ar_audio_btn = gr.Button("🔊", elem_id="small_btn")
                ar_audio_output = gr.Audio(label="", type="filepath")  # ✅ تعريف مخرج الصوت العربي

        with gr.Column():
            with gr.Row():
                pos_output = gr.HTML(
                    "<div style='border:5px solid #FF1493; border-radius:20px; padding:10px; background-color:#FFF0F5; min-height:120px;'>POS Table will appear here...</div>"
                )
                pos_lang = gr.Dropdown(choices=["en", "ar"], value="en", label="", elem_id="small_dropdown")
                pos_button = gr.Button("📝", elem_id="small_btn")

    story_cache = {"text": "", "arabic": ""}

    def generate_and_update(image):
        story = generate_story(image)
        story_cache["text"] = story
        return f"<div style='border:5px solid #FF1493; border-radius:20px; padding:10px; background-color:#E0FFFF;'>{story}</div>"

    image_input.change(generate_and_update, inputs=image_input, outputs=story_output)

    def show_pos(lang):
        text = story_cache["text"] if lang == "en" else story_cache["arabic"]
        return f"<div style='border:5px solid #FF1493; border-radius:20px; padding:10px; background-color:#FFF0F5;'>{analyze_pos_child_friendly(text, lang)}</div>"

    pos_button.click(show_pos, inputs=pos_lang, outputs=pos_output)

    def play_audio():
        return get_audio(story_cache["text"], lang="en")
    gen_audio_btn.click(play_audio, outputs=gen_audio_btn)

    def do_translate():
        story_ar = translate_to_arabic(story_cache["text"])
        story_cache["arabic"] = story_ar
        return f"<div style='border:5px solid #32CD32; border-radius:20px; padding:10px; background-color:#F0FFF0;'>{story_ar}</div>"

    story_ar_output.click(do_translate, outputs=story_ar_output)

    def play_audio_ar():
        return get_audio(story_cache["arabic"], lang="ar")
    ar_audio_btn.click(play_audio_ar, outputs=ar_audio_output)

demo.launch(share=True)
